Jupyter Notebook which opens processed LCLS timescans and calculates the root mean square of the Bragg peak in the region of interest

In [ ]:
### import modules
import h5py
import matplotlib.pyplot as plt
import numpy as np
import itertools
from scipy import signal
from tempfile import TemporaryFile
import matplotlib.gridspec as gridspec


In [ ]:
### import data
ON_HiFluence_on = np.load("/home/jg2364/lcls_lao_analysis/py_nb_makeArrays/008_YES187/timescans/ON/Hi_Fluence/35Array_On")
ON_HiFluence_off = np.load("/home/jg2364/lcls_lao_analysis/py_nb_makeArrays/008_YES187/timescans/ON/Hi_Fluence/35Array_Off")

print(ON_HiFluence_on.shape)
print(ON_HiFluence_off.shape)



In [ ]:
%matplotlib notebook

### region of interest shape
dec_pixel = np.linspace(0, 20, 20)

### number of time delay bins
widths = np.zeros(61)
errors = np.zeros(61)

### intensity curve
curve = np.sum(ON_HiFluence_on[1,:,:], axis=1)/np.max(np.sum(ON_HiFluence_on[1,:,:], axis=1))


curve_part = curve#[np.argmax(curve)-9:np.argmax(curve)+9]
pixel_part = dec_pixel#[np.argmax(curve)-9:np.argmax(curve)+9]

xx = pixel_part#-pixel_part[np.argmax(curve_part)]
yy = curve_part

rms = np.sqrt(np.sum(yy*(xx-(np.sum(xx*yy)/np.sum(yy)))**2) / np.sum(yy))

plt.plot(xx, yy, '-o', color='blue')
plt.xlabel(r"Detector Pixel $\perp$ to S.P.")
plt.ylabel("Normalized Intensity")
print(rms)

rms_array = np.zeros(len(widths))
rms_array_off = np.zeros(len(widths))

for u in range(0, 61, 1):
    # intensity curve projected along y direction of the ROI, for laser on and off
    curve = np.sum(ON_HiFluence_on[u,:,:], axis=1)/np.max(np.sum(ON_HiFluence_on[u,:,:], axis=1))
    curve_off = np.sum(ON_HiFluence_off[u,:,:], axis=1)/np.max(np.sum(ON_HiFluence_off[u,:,:], axis=1))
    
    ### curve (y) and pixel (x) peaks to take the rms of 
    curve_part = curve
    pixel_part = dec_pixel
    curve_part_off = curve_off
    pixel_part = dec_pixel

    ### center the x-axis 
    xx = pixel_part-pixel_part[np.argmax(curve_part)]
    yy = curve_part
    yy_off = curve_part_off
    
    # calculate the root mean square of the peak
    rms = np.sqrt(np.sum(yy*(xx-(np.sum(xx*yy)/np.sum(yy)))**2) / np.sum(yy))
    rms_off = np.sqrt(np.sum(yy_off*(xx-(np.sum(xx*yy_off)/np.sum(yy_off)))**2) / np.sum(yy_off))
    
    rms_array[u] = rms
    rms_array_off[u] = rms_off

### plot the peak and the rms for on and off    
plt.figure()

plt.plot(np.linspace(-2, 20, 61), rms_array, color='r')
plt.plot(np.linspace(-2, 20, 61), rms_array_off, color='b')
plt.xlabel("time delay (ps)")

### save the rms for on and off
with open('35Array_rms_1_on', 'wb') as f:
    np.save(f, rms_array)

with open('35Array_rms_1_off', 'wb') as f:
    np.save(f, rms_array_off)